In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
from pkg.tf_transform import *
from pkg.tf_robot import *
from pkg.constraint import *
from pkg.info import *
from pkg.tf_utils import *
from pkg.rotation_utils import *

In [ ]:
import matplotlib.pyplot as plt
import time

# Load urdf 

In [ ]:
from urdf_parser_py.urdf import URDF
from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE
from pkg.joint_utils import *
urdf_content = URDF.from_xml_file(URDF_PATH)

In [ ]:
link_names = LINK_NAMES
base_name = LINK_NAMES[0]
joint_names = JOINT_NAMES
link_info_list = get_link_info_list(link_names, urdf_content)

# Visualize

In [ ]:
from pkg.ros_rviz import *
pub, joints, rate = get_publisher(JOINT_NAMES)

In [ ]:
import pickle

In [ ]:

with open("tmp.pkl", 'wb') as output:  # Overwrites any existing file.
    pickle.dump(urdf_content, output, pickle.HIGHEST_PROTOCOL)

# Constraint

In [ ]:
from pkg.geometry import *

In [ ]:
def get_adjacent_links(link_name, urdf_content):
    adjacent_links = [link_name]
    for k, v in urdf_content.joint_map.items():
        if v.parent == link_name:
            adjacent_links += [v.child]
        if v.child == link_name:
            adjacent_links += [v.parent]
    return list(set(adjacent_links))

In [ ]:
class ObjectLayer(layers.Layer):
    def __init__(self, gitem, N_sim, *args, **kwargs):
        self.gitem, self.N_sim = gitem, N_sim
        super(ObjectLayer, self).__init__(*args, **kwargs)
        
    def set_frame(self, Toff_list, link_idx_list, N_link):
        self.Toff_list = tf.constant(Toff_list) # (N_sim, 4,4)
        self.link_one_hot = tf.reshape(tf.one_hot(link_idx_list, N_link), (N_sim,N_link,1,1)) # (N_sim, N_link)
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, input=None):
        T_all = input # (N_sim, N_link, 4,4)
        T_act = K.sum(T_all*self.link_one_hot, axis=1) # (N_sim, 4,4)
        T_bo = tf.matmul(T_act, self.Toff_list) # (N_sim, 4,4)
        return T_bo

# Graph

In [ ]:
class GraphModel(tf.keras.Model):
    def __init__(self, robot_info, gitem_list, N_sim, alpha_jc=5, alpha_fc=200):
        super(GraphModel, self).__init__()
        self.alpha_jc = alpha_jc
        self.alpha_fc = alpha_fc
        self.robot_info = robot_info
        self.robot = RobotLayer(
            robot_info.link_info_list, rname = robot_info.rname, dim=N_sim)
        self.joint_constraint = JointConstraintLoss(self.robot)
        self.frame_constraint = FrameConstraintLoss()
        self.robot_base = [robot_info.base_frame]*N_sim
            
        self.object_dict = {}
        self.object_name_list = []
        for gitem in gitem_list:
            self.object_dict[gitem.name] = ObjectLayer(gitem, N_sim)
            self.object_name_list += [gitem.name]
            
    def assign_frame_dict(self, gframeset_list):
        frame_dict = {k: [] for k in self.object_dict.keys()}
        link_dict = {k: [] for k in self.object_dict.keys()}
        for gframeset in gframeset_list:
            for k, gframe in gframeset.items():
                frame_dict[k] += [gframe.Toff]
                link_dict[k] += [self.robot.link_name_list.index(gframe.link_name)]
        for k in frame_dict.keys():
            self.object_dict[k].set_frame(np.array(frame_dict[k]), np.array(link_dict[k]), self.robot.len_Q)
            
    def assign_Q(self, Q):
        self.robot.assign_Q(Q)
            
    def get_Q(self):
        return self.robot.get_Q()

    def call(self, inputs=None):
        T_all = self.robot(self.robot_base)
        Tbo_all = []
        for obj_name in self.object_name_list:
            Tbo_all += [self.object_dict[obj_name](T_all)] #(Nobj,N_sim,4,4)
        Tbo_all = K.stack(Tbo_all, axis=1) #(N_sim,Nobj,4,4)
        return T_all, Tbo_all
    
    def calc_loss(self, T_all, Tbo_all, Qtar, binQ, Ttar, binT):
        jc_loss = self.joint_constraint((Qtar, binQ))
        fc_loss = self.frame_constraint((T_all[:,-1,:,:],Ttar, binT))
        return self.alpha_jc*jc_loss+self.alpha_fc*fc_loss


In [ ]:
N_sim = 50
N_joints = 9
DOF = 6
robot_info = RobotInfo(link_info_list, rname = "rbt1", base_frame=np.identity(4,dtype=np.float32))
gitem_list = [
    GeometryItem(name='box1', gtype=GeoType.BOX, dims=[0.1,0.1,0.1], color=(0,1,0,1), display=True, collision=True),
    GeometryItem(name='tool', gtype=GeoType.BOX, dims=[0.1,0.1,0.1], color=(0,1,0,1), display=True, collision=True),
]
gframe_dict = {"box1":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.5,0,0)), "world"),
               "tool":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.0,0.0,0.1)), "tool0")
              }
marker_list = set_markers(gitem_list, gframe_dict, urdf_content)
graph = GraphModel(robot_info=robot_info, gitem_list=gitem_list, N_sim=N_sim)
optimizer = tf.optimizers.SGD(learning_rate=5e-3)

In [ ]:
Q_ = np.array([(0, 0,)+tuple((np.random.rand(DOF)*2-1)*np.pi)+(0,) for _ in range(N_sim)], dtype=np.float32)
Ttar = SE3(Rot_zyx(0,0,np.pi),(0.5,0,0)).astype(np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
time_vec = []
Q_list = [list(map(lambda x: x.numpy(), graph.get_Q()))]
Tbo_all_list = [np.array([[gframe_dict[gitem.name].Toff for gitem in gitem_list]]*N_sim)]
N_iter = 100
# Run training for the given number of steps.
for _ in range(N_iter):
    # Run the optimization to update W and b values.
    t1 = time.time()
    with tf.GradientTape() as g:
        # Forward pass.
        T_all, Tbo_all= graph(None)
        loss = graph.calc_loss(T_all, Tbo_all, 
                               np.zeros((N_sim, N_joints), dtype='float32'), np.zeros(N_sim, dtype='float32'),
                               np.array([Ttar]*N_sim), np.ones(N_sim, dtype='float32'))
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = graph.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
#     gradients = clip_gradient(gradients, 10)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    t2 = time.time()
    time_vec += [t2-t1]
    Q_list += [list(map(lambda x: x.numpy(), graph.get_Q()))]
    Tbo_all_list += [np.array(Tbo_all)]
print("mean {} ms".format(round(np.mean(time_vec)*1000,2)))
print("total {} ms".format(round(np.sum(time_vec)*1000,2)))
print("loss = {}".format(loss))

In [ ]:
for i_sim in range(1): # N_sim):
    for i_iter in range(1,N_iter+1):
        q = Q_list[i_iter][i_sim]
        gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
        pose_list = [q[2:-1]]
        gframevec_list = [gframevec]
        show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)
        time.sleep(2e-2)

In [ ]:
Q_all = np.array(Q_list)
Q_all_i = np.array([Q_all[i_iter, i_sim,:] for i_iter in range(N_iter)])
plt.plot(Q_all_i)

In [ ]:
plt.plot(Q_all_i[1:,:]-Q_all_i[:-1,:])

In [ ]:
# N_r = 100
# DOF = 6
# robot = RobotLayer(link_info_list, rname = "rb1", base_name=base_name, dim=N_r)
# qvec = []
# Pvec_list = []
# time_list = []
# asg_time_list = []
# for _ in range(100):
#     q = np.array([(0,)+tuple((np.random.rand(DOF)*2-1)*np.pi/3)+(0,) for _ in range(N_r)], dtype=np.float32)
#     qvec += [q]
#     t0 = time.time()
#     robot.assign_Q(q)
#     t01 = time.time()
#     T_all = robot([np.identity(4,dtype=np.float32)]*N_r)
#     t1 = time.time()
#     Pvec = np.array(T_all[:,:,:3,3])
#     Pvec_list += list(Pvec)
#     time_list += [t1-t0]
#     asg_time_list += [t01-t0]
# Pvec_list=np.array(Pvec_list)
# print("assign average: {} ms".format(np.mean(asg_time_list)*1000))
# print("total average: {} ms".format(np.mean(time_list)*1000))
# # for Pvec in Pvec_list:
# #     plt.plot(Pvec[:,0],Pvec[:,1],'-o')
# # plt.axis('equal')

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

In [ ]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.1
training_steps = 2000
batch_size = 50
display_step = 100

# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 256 # 2nd layer number of neurons.

In [ ]:
# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [ ]:

# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:

# Create TF Model.
class NeuralNet(Model):
    # Set layers.
    def __init__(self):
        super(NeuralNet, self).__init__()
        # First fully-connected hidden layer.
        self.fc1 = layers.Dense(n_hidden_1, activation=tf.nn.relu)
        # First fully-connected hidden layer.
        self.fc2 = layers.Dense(n_hidden_2, activation=tf.nn.relu)
        # Second fully-connecter hidden layer.
        self.out = layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
neural_net = NeuralNet()

In [ ]:

# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [ ]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        pred = neural_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = neural_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [ ]:
import time

In [ ]:
time_vec = []
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    t1 = time.time()
    run_optimization(batch_x, batch_y)
    t2 = time.time()
    time_vec += [t2-t1]
    
    if step % display_step == 0:
        pred = neural_net(batch_x, is_training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

In [ ]:
np.mean(time_vec)*100